In [1]:
# Fairness-aware Classifier with Prejudice Remover Regularizer (PR)
import numpy as np
import pandas as pd
pd.options.display.max_columns = 1000



In [2]:
url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
df = pd.read_csv(url)
df.head(6)

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-08-14,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,13009779CF10A,(F3),NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,NaN,1,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,2013-01-27,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,13011511MM10A,(M1),0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,2013-04-14,Risk of Violence,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,8,High,2013-01-13,Risk of Violence,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,1,0,0,2,NaN,NaN,NaN,12014130CF10A,NaN,2013-01-09,76.0,F,arrest case no charge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-03-26,Risk of Violence,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0
5,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,0,1,0,0,0,0.0,2013-11-30 04:50:18,2013-12-01 12:28:56,13022355MM10A,2013-11-30,NaN,0.0,M,Battery,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-11-30,Risk of Violence,1,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0


In [3]:
def remove_missing_records(df: pd.DataFrame, threshold: float = 0.5) -> pd.DataFrame:
    """Remove records with missing values above a threshold."""
    # Get the number of missing values per column
    missing = df.isna().sum()
    # Get the columns with missing values above the threshold
    cols = missing[missing > threshold * df.shape[0]].index
    # Remove the columns
    df = df.drop(cols, axis=1)
    # Remove the rows with missing values
    df = df.dropna()
    return df

def inpute_missing_data(df):
    """Inpute missing data with the mean."""
    # Get the number of missing values per column
    missing = df.isna().sum()
    # Get the columns with missing values
    cols = missing[missing > 0].index
    # Inpute the missing values with the mean
    for col in cols:
        df[col] = df[col].fillna(df[col].mean())
    return df

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """Preprocess the data."""
    # Remove the records with missing values
    df = remove_missing_records(df)
    # Inpute the missing values
    df = inpute_missing_data(df)
    return df

Data Cleaning code

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random

import torch as t
import torch.nn as nn

In [5]:
data = pd.read_csv(url)

In [6]:
data.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,violent_recid,is_violent_recid,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-08-14,Risk of Violence,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26,NaN,1.0,F,Felony Battery w/Prior Convict,1,13009779CF10A,(F3),NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,NaN,1,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,2013-01-27,Risk of Violence,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13,NaN,1.0,F,Possession of Cocaine,1,13011511MM10A,(M1),0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,4,Low,2013-04-14,Risk of Violence,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,8,High,2013-01-13,Risk of Violence,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,0,1,0,0,2,NaN,NaN,NaN,12014130CF10A,NaN,2013-01-09,76.0,F,arrest case no charge,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,Risk of Recidivism,1,Low,2013-03-26,Risk of Violence,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [7]:
data = data[['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count',             
            'days_b_screening_arrest', 'decile_score', 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out']]

In [8]:
data = data.loc[(data['days_b_screening_arrest'] <= 30) & (data['days_b_screening_arrest'] >= -30)]

In [9]:
data = data.loc[data['is_recid'] != -1]

In [10]:
data = data.loc[data['c_charge_degree'] != 'O']

In [11]:
data = data.loc[data['score_text'] != 'N/A']

In [12]:
data["length_of_stay"] = (pd.to_datetime(data['c_jail_out'])-pd.to_datetime(data['c_jail_in'])).dt.days

In [13]:
data = data.drop(columns=['c_jail_in', 'c_jail_out'])

In [14]:
data = data.loc[(data["race"] == "African-American") | (data["race"] == "Caucasian")]

In [15]:
data = data.replace({'race': 'Caucasian'}, 1)
data = data.replace({'race': 'African-American'}, 0)

data = data.replace({'sex': 'Male'}, 1)
data = data.replace({'sex': 'Female'}, 0)

data = data.replace({'age_cat': 'Less than 25'}, 0)
data = data.replace({'age_cat': '25 - 45'}, 1)
data = data.replace({'age_cat': 'Greater than 45'}, 2)

data = data.replace({'c_charge_degree': 'F'}, 0)
data = data.replace({'c_charge_degree': 'M'}, 1)

data = data.replace({'score_text': 'Low'}, 0)
data = data.replace({'score_text': 'Medium'}, 1)
data = data.replace({'score_text': 'High'}, 2)

In [16]:
data.head()

,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,length_of_stay
1,34,0,0,1,0,1,0,-1.0,3,1,1,10
2,24,0,0,0,0,1,4,-1.0,4,1,1,1
6,41,0,1,1,1,1,14,-1.0,6,1,1,6
8,39,1,1,1,0,0,0,-1.0,1,0,0,2
10,27,0,1,1,0,1,0,-1.0,4,0,0,1


In [17]:
X = data.drop(columns=["two_year_recid"])
y = data["two_year_recid"]

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
X_train_a = X_train[(X_train['race'] == 0)]
X_train_c = X_train[(X_train['race'] == 1)]
y_train_a = y_train[(X_train['race'] == 0)]
y_train_c = y_train[(X_train['race'] == 1)]

In [20]:
X_test_a = X_test[(X_test['race'] == 0)]
X_test_c = X_test[(X_test['race'] == 1)]
y_test_a = y_test[(X_test['race'] == 0)]
y_test_c = y_test[(X_test['race'] == 1)]

In [21]:
X_train_a = t.tensor(np.array(X_train_a)).to(t.float32)
y_train_a = t.from_numpy(np.array(y_train_a).astype('float32')).reshape(X_train_a.shape[0], 1)
X_train_c = t.tensor(np.array(X_train_c)).to(t.float32)
y_train_c = t.from_numpy(np.array(y_train_c).astype('float32')).reshape(X_train_c.shape[0], 1)

X_test_a = t.tensor(np.array(X_test_a)).to(t.float32)
y_test_a = t.from_numpy(np.array(y_test_a).astype('float32')).reshape(X_test_a.shape[0], 1)
X_test_c = t.tensor(np.array(X_test_c)).to(t.float32)
y_test_c = t.from_numpy(np.array(y_test_c).astype('float32')).reshape(X_test_c.shape[0], 1)

In [22]:
class LogisticRegression(nn.Module):
    def __init__(self,data):
        super(LogisticRegression, self).__init__()
        self.w = nn.Linear(data.shape[1], out_features=1, bias=True)
        self.sigmod = nn.Sigmoid()
    def forward(self, x):
        w = self.w(x)
        output = self.sigmod(w)
        return output

In [23]:
def metrics_cal(Model_a,Model_c, X_a, y_a, X_c, y_c):
    y_pred_a = (Model_a(X_a) >= 0.5)
    y_pred_c = (Model_c(X_c) >= 0.5)
    accuracy_a  = t.sum(y_pred_a.flatten() == y_a.flatten()) / y_a.shape[0]
    accuracy_c  = t.sum(y_pred_c.flatten() == y_c.flatten()) / y_c.shape[0]
    accuracy = (accuracy_a + accuracy_c) / 2
    calibration = t.abs(accuracy_a - accuracy_c)
    return round(accuracy.item(),4), round(calibration.item(),4)

In [24]:
class PRLoss():
    def __init__(self, eta=1.0):
        super(PRLoss, self).__init__()
        self.eta = eta       
        
    def forward(self,output_a,output_c):
        N_a = t.tensor(output_a.shape[0])
        N_c   = t.tensor(output_c.shape[0])
        Dxisi = t.stack((N_a,N_c),axis=0)
        # Pr[y|s]
        y_pred_a = t.sum(output_a)
        y_pred_c   = t.sum(output_c)
        P_ys = t.stack((y_pred_a,y_pred_c),axis=0) / Dxisi
        # Pr[y]
        P = t.cat((output_a,output_c),0)
        P_y = t.sum(P) / (X_train_a.shape[0]+X_train_c.shape[0])
        # P(siyi)
        P_s1y1 = t.log(P_ys[1]) - t.log(P_y)
        P_s1y0 = t.log(1-P_ys[1]) - t.log(1-P_y)
        P_s0y1 = t.log(P_ys[0]) - t.log(P_y)
        P_s0y0 = t.log(1-P_ys[0]) - t.log(1-P_y)
        # PI
        PI_s1y1 = output_a * P_s1y1
        PI_s1y0 =(1- output_a) * P_s1y0
        PI_s0y1 = output_c * P_s0y1
        PI_s0y0 = (1- output_c )* P_s0y0
        PI = t.sum(PI_s1y1) + t.sum(PI_s1y0) + t.sum(PI_s0y1) + t.sum(PI_s0y0)
        PI = self.eta * PI
        return PI

In [25]:
class PRLR():
  
    def __init__(self, eta=0.0, epochs=100, lr = 0.01):
        super(PRLR, self).__init__()
        self.eta = eta
        self.epochs = epochs
        self.lr = lr
        
    def fit(self,X_train_a,y_train_a,X_train_c,y_train_c,
            X_test_a, y_test_a, X_test_c, y_test_c):     
        model_a = LogisticRegression(X_train_a)
        model_c = LogisticRegression(X_train_c)
        criterion = nn.BCELoss(reduction='sum')
        PI = PRLoss(eta=self.eta)
        epochs = self.epochs
        optimizer = t.optim.Adam(list(model_a.parameters())+ list(model_c.parameters()), self.lr, weight_decay=1e-5)
        
        for epoch in range(epochs):
            model_a.train()
            model_c.train()
            optimizer.zero_grad()
            output_a = model_a(X_train_a)
            output_c = model_c(X_train_c)
            logloss = criterion(output_a, y_train_a)+ criterion(output_c, y_train_c)
            PIloss = PI.forward(output_a,output_c)
            loss = PIloss +logloss
            loss.backward()
            optimizer.step()
            
        model_a.eval()
        model_c.eval()
        accuracy, calibration = metrics_cal(model_a,model_c,X_test_a, y_test_a, X_test_c, y_test_c)
        return accuracy, calibration

In [26]:
PR = PRLR(eta = 1.0, epochs = 50, lr = 0.01)
random.seed(2)
PR.fit(X_train_a,y_train_a,X_train_c,y_train_c, 
       X_test_a, y_test_a, X_test_c, y_test_c)

(0.5626, 0.0398)

### LFR algorithm


#### Helper functions

In [27]:
import numpy as np
import pandas as pd
from scipy import optimize

In [28]:
def distances(X, V, alpha):
  N,D = X.shape
  K,D = V.shape
  dists = np.zeros((N, D))
  for i in range(N):
    for d in range(D):
      for k in range(K):    
        dists[i, k] += (X[i, d] - V[k, d]) * (X[i, d] - V[k, d]) * alpha[d]
    return dists

In [29]:
def M_nks(dists,K):
  N,D = dists.shape
  upper = np.zeros((N,K))
  lower = np.zeros(N)
  M_nks = np.zeros((N,K))
  for i in range(N):
    for j in range(K):
      upper[i,j] = np.exp(-dists[i,j])
      lower[i] += upper[i,j]
    for j in range(K):
      if lower[i]:
        M_nks[i,j] = upper[i,j]/lower[i]
      else:
        M_nks[i, j] = upper[i, j] / 1e-6
  return M_nks

In [30]:
def M_ks (X,M_nks):
  N,K = M_nks.shape
  M_ks = np.zeros(K)
  for i in range(K):
    for j in range(N):
      M_ks[i] += M_nks[j,i]
    M_ks[i] = M_ks[i]/N
  return M_ks

In [31]:
def X_hat(X,V,M_nks):
  N,D = X.shape
  K,D = V.shape
  X_hat = np.zeros((N,D))
  for i in range(N):
    for d in range(D):
      for j in range(K):
        X_hat[i,d] += M_nks[i,j]*V[j,d]
  return X_hat


In [32]:
def Y_hat(M_nks,W):
  N,K = M_nks.shape
  Y_hat = np.zeros(N)
  for i in range(N):
    for j in range(K):
      Y_hat[i] += M_nks[i,j]*W[j]
    Y_hat[i] = 1e-6 if Y_hat[i] <= 0 else Y_hat[i]
    Y_hat[i] = 0.999 if Y_hat[i] >= 1 else Y_hat[i]
  return Y_hat

In [33]:
def l_x(X,X_hat):
  l_x = 0
  N,D = X.shape
  for i in range(N):
    for d in range(D):
      l_x += (X[i,d]-X_hat[i,d])**2
  return l_x

In [34]:
def l_y(Y,Y_hat):
  l_y = 0
  N = len(Y)
  for i in range(N):
    l_y += -Y[i]*np.log(Y_hat[i]) - (1-Y[i])*(np.log(1-Y_hat[i]))
  return l_y

In [35]:
def LFR(params,X_sen,X_unsen,Y_sen,Y_unsen, k=10, A_x = 1e-4, A_y = 0.1, A_z = 1000, result=0):
  LFR.iters+=1
  N1,P = X_sen.shape
  N0,_ = X_unsen.shape

  alpha0 = params[:P]
  alpha1 = params[P: 2 * P]
  W = params[2 * P: (2 * P) + k]
  V = np.matrix(params[(2 * P) + k:]).reshape((k, P))
  dists_sen = distances(X_sen, V, alpha1)
  dists_unsen = distances(X_unsen, V, alpha0)
  M_nks_sen = M_nks(dists_sen,k)
  M_nks_unsen = M_nks(dists_unsen,k)
  M_ks_sen = M_ks(X_sen,M_nks_sen)
  M_ks_unsen = M_ks(X_unsen,M_nks_unsen)

  X_hat_sen = X_hat(X_sen,V,M_nks_sen)
  X_hat_unsen = X_hat(X_unsen,V,M_nks_unsen)
  
  L_x_1 =  l_x(X_sen,X_hat_sen)
  L_x_0 = l_x(X_unsen,X_hat_unsen)

  Y_hat_sen = Y_hat(M_nks_sen,W)
  Y_hat_unsen = Y_hat(M_nks_unsen,W)

  L_y_1 = l_y(Y_sen,Y_hat_sen)
  L_y_0 = l_y(Y_unsen,Y_hat_unsen)

  L_X = L_x_1+L_x_0
  L_Y = L_y_1+L_y_0

  L_z = 0
  
  for i in range(k):
    L_z += abs(M_ks_sen[i]-M_ks_unsen[i])

  L_obj = A_z*L_z + A_x*L_X + A_y*L_Y

  if LFR.iters % 250 == 0:
    print(LFR.iters, L_obj)
  
  if result:
    return Y_hat_sen, Y_hat_unsen, M_nks_sen , M_nks_unsen
  else:
    return L_obj


#### Data wrangler

In [36]:
X = data.drop(columns=["two_year_recid"])
y = data["two_year_recid"]

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train_a = X_train[(X_train['race'] == 0)]
X_train_c = X_train[(X_train['race'] == 1)]
y_train_a = y_train[(X_train['race'] == 0)]
y_train_c = y_train[(X_train['race'] == 1)]
X_test_a = X_test[(X_test['race'] == 0)]
X_test_c = X_test[(X_test['race'] == 1)]
y_test_a = y_test[(X_test['race'] == 0)]
y_test_c = y_test[(X_test['race'] == 1)]
#X_train_a = X_train_a.drop(columns=["race"])
#X_train_c = X_train_c.drop(columns=["race"])
#X_test_a = X_test_a.drop(columns=["race"])
#X_test_c = X_test_c.drop(columns=["race"])
X_train_a = t.tensor(np.array(X_train_a)).to(t.float32)
y_train_a = t.from_numpy(np.array(y_train_a).astype('float32')).reshape(X_train_a.shape[0], 1)
X_train_c = t.tensor(np.array(X_train_c)).to(t.float32)
y_train_c = t.from_numpy(np.array(y_train_c).astype('float32')).reshape(X_train_c.shape[0], 1)

X_test_a = t.tensor(np.array(X_test_a)).to(t.float32)
y_test_a = t.from_numpy(np.array(y_test_a).astype('float32')).reshape(X_test_a.shape[0], 1)
X_test_c = t.tensor(np.array(X_test_c)).to(t.float32)
y_test_c = t.from_numpy(np.array(y_test_c).astype('float32')).reshape(X_test_c.shape[0], 1)

#### LFR Algorithm Implementation


In [38]:
LFR.iters = 0
# Initialize V_ks
#Number of prototypes:K

k=10
D = 11
rez = np.random.uniform(size=D * 2 + k + D * k)

bnd = []
for i, k2 in enumerate(rez):
    if i < D * 2 or i >= D * 2 + k:
        bnd.append((None, None))
    else:
        bnd.append((0, 1))

rez = optimize.fmin_l_bfgs_b(LFR, x0=rez, epsilon=1e-3, 
                          args=(X_train_a, X_train_c, 
                                y_train_a, y_train_c, k, 1e-4,
                                0.1, 1000, 0),
                          bounds = bnd, approx_grad=True, maxfun=2000, 
                          maxiter=2000)

w = rez[0][D*2:D*2+k]
v = rez[0][D*2+k:].reshape(k,D)

KeyboardInterrupt: ignored

In [39]:
y_test_a = y_test_a.flatten()
y_test_a = list(y_test_a)
y_test_c = y_test_c.flatten()
y_test_c = list(y_test_c)
y_test= y_test_a + y_test_c
y_test = np.array(y_test)


In [40]:
LFR.iters=0
yhat_a, yhat_c, Mnk_a,Mnk_c = LFR(rez[0],
                                    X_train_a,
                                    X_train_c,
                                    y_train_a,
                                    y_train_c,10, 1e-4, 0.1, 1000, result=1)

yhat_a_test, yhat_c_test, Mnk_a_test,Mnk_c_test = LFR(rez[0],
                                    X_test_a,
                                    X_test_c,
                                    y_test_a,
                                    y_test_c,10, 1e-4, 0.1, 1000, result=1)


IndexError: ignored

In [ ]:
Y_pred_a = [1 if y >= 0.5 else 0 for y in yhat_a_test]
Y_pred_c = [1 if y >= 0.5 else 0 for  y in yhat_c_test]

In [ ]:
def metrics_cal1( y_pred_a, y_a, y_pred_c, y_c):
    #y_pred_a = (1 if i >= 0.5 else 0 for i in y_pred_a)
    #y_pred_c = (1 if i >= 0.5 else 0 for  i in Y_pred_c)
    accuracy_a  = np.sum(y_pred_a == y_a.flatten()) / y_a.shape[0]
    accuracy_c  = np.sum(y_pred_c == y_c.flatten()) / y_c.shape[0]
    accuracy = (accuracy_a + accuracy_c) / 2
    discrimination = abs(sum(y_pred_a)/len(y_pred_a)-sum(y_pred_c)/len(y_pred_c))
    calibration = abs(accuracy_a - accuracy_c)
    return round(accuracy.item(),4), round(calibration.item(),4), round(discrimination.item(),4)

In [ ]:
results = metrics_cal1(np.array(Y_pred_a), np.array(y_test_a), np.array(Y_pred_c), np.array(y_test_c))
results

# ALG 5

In [42]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

## Prejudice

**Prejudice** means a statistical dependence between a sensitive variable,
$S$, and the target variable, $Y$, or a non-sensitive variable, $X$.

There are three types of prejudices:
## Direct prejudice

Direct prejudice is the use of a sensitive variable in a prediction model.

To eliminate direct prejudice, we can remove the sensitive variable from the model.


## Indirect prejudice

Indirect prejudice is statistical dependence between a sensitive variable and a target variable.

To remove this indirect prejudice, we must use a prediction model that satisfies the condition $Y \perp\!\!\!\perp \ S$.

We can quantify the degree of indirect prejudice using the following equation where $PI$ refers to the (indirect) prejudice index and $\cal{D}$ is the data set.

$$\text{PI} = \sum_{(y, s) \in \cal{D}}  \hat{\text{Pr}}[y, s] \ln \frac{\hat{\text{Pr}}[y, s]}{\hat{\text{Pr}}[y]\hat{\text{Pr}}[s]}$$

The application of the normalization technique for mutual information leads to a _normalized prejudice index_ (NPI)

$$\text{NPI} = \frac{\text{PI}}{\sqrt{\text{H}(Y)\text{H}(S)}}$$

where $\text{H}(\ \cdot\ )$ is the entropy function.

## Latent prejudice
Latent prejudice is a statistical dependence between a sensitive variable, $S$, and a non-sensitive variable, $X$.

Removal of potential prejudice is achieved by making $X$ and $Y$ independent from $S$ simultaneously.



## Underestimation

Underestimation is the state in which a learned model is not fully converged due to the finiteness of the size of a training data set.

Despite that a prediction model without indirect prejudice can learn to make a fair determination, this is only the case if we have an "infinitely large" training data set. In general, training sets are finite and limited to small quantities of data, hence the model could output even more unfair determinations than that observed in the training sample distribution.

To quantify the degree of underestimation, we assess the resultant difference between the training sample distribution over $\cal{D}$, $\tilde{\text{Pr}}$ using the underestimation index (UEI) which is calculated using the Hellinger distance:

$$\text{UEI} = \sqrt{\frac{1}{2}\sum_{(y, s) \in \cal{D}} \left(\sqrt{\tilde{\text{Pr}}[y, s]} - \sqrt{\hat{\text{Pr}}[y, s]}\right)^2} = \sqrt{1 - \sum_{(y, s) \in \cal{D}} \sqrt{\hat{\text{Pr}}[Y, S]\tilde{\text{Pr}}[Y, S]}}$$

where $\hat{\text{Pr}}$ is the distribution of the learned model.

## Negative Legacy

Negative legacy is unfair sampling or labeling in the training data. 

For example, if a bank has been refusing credit to minority people without
assessing them, the records of minority people are less sampled in a training data
set.

## General Framework

Given a training data set $\cal{D} = $ $\{(y, \textbf{x}, s)\}$, we can define the following terms:

- $\cal{M}$ $[ Y |X, S; \mathbb{\Theta}]$ conditional probability of a class given non-sensitive and sensitive features model.
- $\mathbb{\Theta}$ set of model parameters. These parameters are estimates based on the maximum likelihood principle:
$$\cal{L}(\cal{D}, \mathbb{\Theta}) = \sum_{(y_i, \textbf{x}_i, s_i) \in \cal{D}} \ln \cal{M} \ [y_i|\textbf{x}_i, s_i;\mathbb{\Theta}].$$

For the optimization process, we use two types of regularizers, the $L_2$ regularizer $||\mathbb{\Theta}||_2^2$ and a second regularizer $R(\cal{D}, \mathbb{\Theta})$, introduced to enforce fair classification. After applying both regularizing techniques, are objective function becomes:
$$-\cal{L}(\cal{D}, \mathbb{\Theta}) + \eta{} \text{R}(\cal{D}, \mathbb{\Theta}) + \frac{\lambda}{2} ||\mathbb{\Theta}||_2^2.$$

## Prejudice Remover

A _prejudice remover_ regularizer directly tries to reduce the prejudice index and is denoted by $\text{R}_{\text{PR}}$. Recall that the prejudice index is defined as

$$\text{PI} = \sum_{Y, S}  \hat{\text{Pr}}[Y, S] \ln \frac{\hat{\text{Pr}}[Y, S]}{\hat{\text{Pr}}[Y]\hat{\text{Pr}}[S]}$$

where

$$\hat{\text{Pr}}[y|s_i] \approx \frac{\sum_{(\textbf{x}_i, s_i) \in {\cal{D}} \text{ s.t. } s_i = s}{\cal{M}}[y|\textbf{x}_i, s; \mathbb{\Theta}]}{|\left\{(\textbf{x}_i, s_i) \in {\cal{D}} \text{ s.t. } s_i = s \right\}|}.$$

$$\hat{\text{Pr}}[y] \approx \frac{\sum_{(\textbf{x}_i, s_i) \in {\cal{D}}}{\cal{M}}[y|\textbf{x}_i, s_i; \mathbb{\Theta}]}{|{\cal{D}}|}.$$

And the prejudice remover regularizer $\text{R}_{\text{PR}}({\cal{D}}, \mathbb{\Theta})$ is defined as

$$\sum_{(\textbf{x}_i, s_i) \in {\cal{D}}}\sum_{y\in\{0, 1\}}{\cal{M}}[y|\textbf{x}_i, s_i;\mathbb{\Theta}]\ln\frac{\hat{\text{Pr}}[y|s_i]}{\hat{\text{Pr}}[y]}$$

This regularizer becomes increasingly large as a class $y$ becomes more likely to be predicted for a sensitive group $s$ than for the entire population, thus making the overall model is influenced less by the sensitive variables.


In [49]:
set(df["race"])

{'African-American',
 'Asian',
 'Caucasian',
 'Hispanic',
 'Native American',
 'Other'}

In [51]:
def remove_missing_records(df: pd.DataFrame, threshold: float = 0.5) -> pd.DataFrame:
    """Remove records with missing values above a threshold."""
    # Get the number of missing values per column
    missing = df.isna().sum()
    # Get the columns with missing values above the threshold
    cols = missing[missing > threshold * df.shape[0]].index
    # Remove the columns
    df = df.drop(cols, axis=1)
    # Remove the rows with missing values
    df = df.dropna()
    return df

def inpute_missing_data(df):
    """Inpute missing data with the mean."""
    # Get the number of missing values per column
    missing = df.isna().sum()
    # Get the columns with missing values
    cols = missing[missing > 0].index
    # Inpute the missing values with the mean
    for col in cols:
        df[col] = df[col].fillna(df[col].mean())
    return df

def encode_categorical_features(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """Encode categorical features."""
    # Get the categorical features
    categorical = df.loc[:, columns].select_dtypes(include="object").columns
    # Encode the categorical features
    for col in categorical:
        df[col] = pd.factorize(df[col])[0]
    return df

def parse_dates(df: pd.DataFrame) -> pd.DataFrame:
    """Parse the dates."""
    # Convert the dates to datetime objects
    date_columns = df.columns.str.endswith("_date")
    for col in df.loc[:, date_columns].columns:
        df[col] = pd.to_datetime(df[col])

    return df

def preprocess_data(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    """Preprocess the data."""
    # Remove the records with missing values
    df = remove_missing_records(df)
    # Inpute the missing values
    df = inpute_missing_data(df)

    df_a = df[df["race"] == "African-American"]
    df_c = df[df["race"] == "Caucasian"]


    # Encode the categorical features
    df_a = encode_categorical_features(df, columns)
    df_c = encode_categorical_features(df, columns)
    # Parse the dates
    df_a = parse_dates(df)
    df_c = parse_dates(df)
    return df_a, df_c

In [52]:
columns = ["sex", "race", "age_cat", "c_charge_degree", "score_text"]
df_a, df_c = preprocess_data(df, columns)

In [53]:
ns_feature_columns = [
    'age', 'c_charge_degree', 'age_cat',
    'score_text', 'sex', 'priors_count', 'days_b_screening_arrest',
    'decile_score', 'is_recid', 'two_year_recid'
]

feature_columns = ns_feature_columns + ["race"]

In [56]:
X_a = df_a[feature_columns]
y_a = df_a["two_year_recid"]

X_c = df_c[feature_columns]
y_c = df_c["two_year_recid"]

In [57]:
labels_a = tf.cast(df_a["two_year_recid"],  dtype = tf.float32)
sensitive_a = tf.cast(df_a["race"],  dtype = tf.float32)
non_sensitive_a = tf.cast(df_a[ns_feature_columns], dtype=tf.float32)

labels_c = tf.cast(df_c["two_year_recid"],  dtype = tf.float32)
sensitive_c = tf.cast(df_c["race"],  dtype = tf.float32)
non_sensitive_c = tf.cast(df_c[ns_feature_columns], dtype=tf.float32)

In [59]:
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_a, y_a, test_size = 0.2)

X_train_a = tf.cast((X_train_a - np.mean(X_train_a)) / np.std(X_train_a), dtype = tf.float32)
X_test_a = tf.cast((X_test_a - np.mean(X_test_a)) / np.std(X_test_a), dtype = tf.float32)

y_train_a = tf.cast(y_train_a, tf.bool)
y_test_a = tf.cast(y_test_a, tf.bool)

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_c, y_c, test_size = 0.2)

X_train_c = tf.cast((X_train_c - np.mean(X_train_c)) / np.std(X_train_c), dtype = tf.float32)
X_test_c = tf.cast((X_test_c - np.mean(X_test_c)) / np.std(X_test_c), dtype = tf.float32)

y_train_c = tf.cast(y_train_c, tf.bool)
y_test_c = tf.cast(y_test_c, tf.bool)


In [156]:
D_a = tf.data.Dataset.from_tensor_slices(
    (labels_a, non_sensitive_a, sensitive_a)
)


train_size_a = X_train_a.shape[0]

train_dataset_a = D_a.take(train_size_a)
test_dataset_a = D_a.skip(train_size_a).take(df_a.shape[0] - train_size_a)

D_c = tf.data.Dataset.from_tensor_slices(
    (labels_c, non_sensitive_c, sensitive_c)
)


train_size_c = X_train_c.shape[0]

train_dataset_c = D_c.take(train_size_c)
test_dataset_c = D_c.skip(train_size_c).take(df_c.shape[0] - train_size_c)

## Model definition

In [159]:
base_model_a = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape = (11)),
    tf.keras.layers.Dense(
        units = 1,
        activation = "sigmoid"
    )
])

base_model_a.compile(
    optimizer = "adam",
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy()
)
base_model_a.fit(X_train_a, y_train_a)


base_model_c = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape = (11)),
    tf.keras.layers.Dense(
        units = 1,
        activation = "sigmoid"
    )
])

base_model_c.compile(
    optimizer = "adam",
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = tf.keras.metrics.BinaryAccuracy()
)
base_model_c.fit(X_train_c, y_train_c)

147/147 [==============================] - 1s 3ms/step - loss: 0.6398 - binary_accuracy: 0.6614


In [160]:
y_pred_a = base_model_a.predict(X_train_a)
y_pred_c = base_model_c.predict(X_train_c)

147/147 [==============================] - 0s 1ms/step


In [164]:
def conditional_density(model, y, s, D, preds):
    """Probability of label given sensitive variable."""
    obs = 0
    n = 0.0001

    for i, (_, xi, si) in enumerate(D):
        c = tf.cond(tf.math.equal(si, s), lambda: 1.0, lambda: 0.0)
        # x = tf.concat([xi, si], 0)
        try:
            p = preds[i]
        except:
            p = 0
        obs += p * c
        n += c

    return obs / n

def density(model, y, D, preds):
    """Probability of label."""
    obs = 0
    n = len(D)

    for i, (_, xi, si) in enumerate(D):
        # x = tf.concat([xi, si], 0)
        # xx = tf.reshape(tf.cast(xi, dtype = tf.float32), [-1, 1])
        # ss = tf.reshape(tf.cast(si, dtype = tf.float32), [1,1])
        # # x = tf.concat([xx, ss], 1)
        try:
            p = preds[i]
        except:
            p = 0
        obs += p

    return obs / n


def prejudice_remover_regularizer(model, D, preds):
    """Prejudice remover regularizer."""
    result = 0
    for _, xi, si in D:
        for y in [0, 1]:
            
            y = tf.cast(y, dtype=tf.float32)
            p1 = conditional_density(model, y, si, D, preds)
            p2 = density(model, y, D, preds)
            q = p1 / p2
            log_odds = np.log(q) 
            result += log_odds * preds
            
    return result

In [162]:
class FairnessModel(tf.keras.Model):
    """Fairness model."""
    def __init__(self, base_model, D, num_features, preds):
        super(FairnessModel, self).__init__()
        
        self.eta = tf.Variable(0.01)
        self.lam = tf.Variable(0.01)

        self.preds = preds

        # self.model = BinaryLogisticRegression(num_features + num_sensitive)
        self.base_model = base_model
        self.D = D  
        self.R_pr = prejudice_remover_regularizer

        self.loss =  tf.nn.sigmoid_cross_entropy_with_logits
        self.acc_metric = tf.keras.metrics.BinaryAccuracy()

    def train_step(self, data):
        """Call the model."""
        x, y = data

        with tf.GradientTape() as tape:
            model_output = self.base_model(x)

            # Prejudice remover regularizer
            R_pr = self.R_pr(self.base_model, self.D, self.preds)
            regularizer = self.eta * R_pr

            ws = self.base_model.get_weights()

            # l2 regularization
            l2 = self.lam * tf.reduce_sum(tf.square(ws))

            log_model_output = tf.math.log(model_output)
            y_pred = tf.reduce_sum(log_model_output) + regularizer + l2

            loss = self.loss(y_pred, y)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Compute our own metrics
        self.acc_metric.update_state(y, y_pred)
        return {"Binary Accuracy": self.acc_metric.result()}

    @property
    def metrics(self):
        return [self.acc_metric]

    

In [ ]:
model_a = FairnessModel(
    base_model_a,
    train_dataset_a,
    non_sensitive_a.shape[1],
    y_pred_a
)
model_a.compile(optimizer = "adam")
model_a.fit(X_train_a, y_train_a, epochs = 1)

model_c = FairnessModel(
    base_model_c,
    train_dataset_c,
    non_sensitive_c.shape[1],
    y_pred_c
)
model_c.compile(optimizer = "adam")
model_c.fit(X_train_c, y_train_c, epochs = 1)

In [133]:
def metrics_cal2( y_pred_a, y_a, y_pred_c, y_c):
    #y_pred_a = (1 if i >= 0.5 else 0 for i in y_pred_a)
    #y_pred_c = (1 if i >= 0.5 else 0 for  i in Y_pred_c)
    accuracy_a  = np.sum(y_pred_a == y_a) / y_a.shape[0]
    accuracy_c  = np.sum(y_pred_c == y_c) / y_c.shape[0]
    accuracy = (accuracy_a + accuracy_c) / 2
    discrimination = abs(sum(y_pred_a)/len(y_pred_a)-sum(y_pred_c)/len(y_pred_c))
    calibration = abs(accuracy_a - accuracy_c)
    return round(accuracy.item(),4), round(calibration.item(),4), round(discrimination.item(),4)

In [139]:
y_pred_a = model_a.predict(X_test_a)
y_pred_c = model_c.predict(X_test_c)
results = metrics_cal2(y_pred_a, y_test_a, y_pred_c, y_test_c)

37/37 [==============================] - 0s 4ms/step


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [141]:
with open('/content/drive/My Drive/results_FRL.txt', 'w') as f:
    np.savetxt(f, results)